# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create Weather DataFrame

csv_path = os.path.join("..", "WeatherPy", "weather_data_output.csv")
weather_df = pd.read_csv(csv_path)
weather_df.dropna()
del weather_df ["Unnamed: 0"]
weather_df.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Chui,UY,2020-03-15 23:40:37,-33.70,-53.46,100,71,64.00,6.35
1,Bhatkal,IN,2020-03-15 23:40:39,13.97,74.57,9,75,76.05,6.11
2,Pangai,TO,2020-03-15 23:40:41,-19.80,-174.35,90,79,84.20,8.05
3,Camacha,PT,2020-03-15 23:40:43,33.08,-16.33,40,72,62.60,13.87
4,Lompoc,US,2020-03-15 23:41:06,34.64,-120.46,90,81,57.20,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
import os
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations and Fill NaN values and convert to float
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Store Humidity values and Fill NaN values and convert to float
humidity = weather_df["Humidity (%)"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create Filtered Weather DataFrame 
ideal_weather_df = pd.DataFrame(weather_df)

# Fit Ideal Weather Conditions based on Cloudiness, Max Temp and Wind Speed
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Cloudiness (%)"] == 0)]                                               
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp (F)"] < 80) & (ideal_weather_df["Max Temp (F)"] > 70)]
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Wind Speed (mph)"] < 10)]    
                                              
# Drop any rows with null values and reset index
ideal_weather_df.dropna()
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df["index"]
ideal_weather_df                              

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Taoudenni,ML,2020-03-15 23:44:05,22.68,-3.98,0,10,73.33,9.35
1,Sagaing,MM,2020-03-15 23:49:54,21.88,95.98,0,28,73.24,8.10
2,Alice Springs,AU,2020-03-15 23:51:30,-23.70,133.88,0,18,73.40,9.17
3,Yuli,NG,2020-03-15 23:52:51,9.70,10.27,0,13,72.77,6.11
4,São João da Barra,BR,2020-03-15 23:54:04,-21.64,-41.05,0,83,78.80,6.93
5,Neuquén,AR,2020-03-16 00:00:59,-38.95,-68.06,0,27,77.00,4.70
6,Najrān,SA,2020-03-16 00:01:09,17.49,44.13,0,37,71.60,1.72


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create New Dataframe 
hotel_df = ideal_weather_df

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,Taoudenni,ML,2020-03-15 23:44:05,22.68,-3.98,0,10,73.33,9.35,
1,Sagaing,MM,2020-03-15 23:49:54,21.88,95.98,0,28,73.24,8.10,
2,Alice Springs,AU,2020-03-15 23:51:30,-23.70,133.88,0,18,73.40,9.17,
3,Yuli,NG,2020-03-15 23:52:51,9.70,10.27,0,13,72.77,6.11,
4,São João da Barra,BR,2020-03-15 23:54:04,-21.64,-41.05,0,83,78.80,6.93,
5,Neuquén,AR,2020-03-16 00:00:59,-38.95,-68.06,0,27,77.00,4.70,
6,Najrān,SA,2020-03-16 00:01:09,17.49,44.13,0,37,71.60,1.72,


In [7]:
hotel_names = []


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # geocoordinates
    target_coordinates = f"{latitude},{longitude}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "Hotel"
    
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}    
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    vacation_response = requests.get(base_url, params=params)

    # convert response to json
    places_data = vacation_response.json()

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    # extract results
    results = places_data['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 0: Taoudenni.
Missing field/result... skipping.
Retrieving Results for Index 1: Sagaing.
Closest hotel is Top Royal Hotel.
Retrieving Results for Index 2: Alice Springs.
Closest hotel is DoubleTree by Hilton Hotel Alice Springs.
Retrieving Results for Index 3: Yuli.
Missing field/result... skipping.
Retrieving Results for Index 4: São João da Barra.
Closest hotel is Pousada Kactus.
Retrieving Results for Index 5: Neuquén.
Closest hotel is Hilton Garden Inn Neuquen.
Retrieving Results for Index 6: Najrān.
Missing field/result... skipping.


In [8]:
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,Taoudenni,ML,2020-03-15 23:44:05,22.68,-3.98,0,10,73.33,9.35,
1,Sagaing,MM,2020-03-15 23:49:54,21.88,95.98,0,28,73.24,8.10,Top Royal Hotel
2,Alice Springs,AU,2020-03-15 23:51:30,-23.70,133.88,0,18,73.40,9.17,DoubleTree by Hilton Hotel Alice Springs
3,Yuli,NG,2020-03-15 23:52:51,9.70,10.27,0,13,72.77,6.11,
4,São João da Barra,BR,2020-03-15 23:54:04,-21.64,-41.05,0,83,78.80,6.93,Pousada Kactus
5,Neuquén,AR,2020-03-16 00:00:59,-38.95,-68.06,0,27,77.00,4.70,Hilton Garden Inn Neuquen
6,Najrān,SA,2020-03-16 00:01:09,17.49,44.13,0,37,71.60,1.72,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))